# Libraries

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
from collections import Counter

from preprocessing import preprocess, substitute_col_by_dict

import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

(with help of Paris Opendata dataset)

# Bottin Data

In [3]:
Bottins = pd.read_csv("data/strict_addressing.csv")
Bottins["Rue"] = Bottins["Rue"].fillna("")
Bottins = preprocess(Bottins, "Rue", "street")

Bottins = Bottins.rename(columns={"Rue":"rue","street":"rue_processed","Nom":"nom","Métier":"metier","Numéro": "numero", "annee":"year"})

In [4]:
Bottins.head(10)

,Unnamed: 0,page,row,nom,metier,rue,numero,year,rue_processed
0,bpt6k6282019m,144,0,Aaron,bronzes,passage Choiseal,72 et 74.,1855,passage choiseal
1,bpt6k6282019m,144,1,Aaron (Mic.),manuf. de porcelaines,Bondy,30.,1855,bondy
2,bpt6k6282019m,144,3,Abadie,architecte,Provence,7.,1855,provence
3,bpt6k6282019m,144,5,Abadie,tabac et estamin.,Ménilmontant,158.,1855,menilmontant
4,bpt6k6282019m,144,6,Abanse,instituteur,Sts-Pères,30.,1855,sts peres
5,bpt6k6282019m,144,7,Abat,rentier,Isly,10.,1855,isly
6,bpt6k6282019m,144,13,Abault et Coudray,charpentiers,Corbeau,23.,1855,corbeau
7,bpt6k6282019m,144,14,Abault (Paul),libraire,quai des Angustins,9.,1855,quai des angustins
8,bpt6k6282019m,144,15,Abavid,vins,Beaujolais-da-Temple,7.,1855,beaujolais da temple
9,bpt6k6282019m,144,16,Abazaer (Are),cristaux et porcelaines,Pei.Ecuries,26.,1855,pei.ecuries


## prefix substitution

#### create a prefix dictionary 


In [5]:
voies_raw = pd.read_csv("data/opendata_voie_paris.csv", sep=";")
#keep only columns that might be useful further on
voies = voies_raw.copy()[["N_SQ_VO", "L_VOIE", "L_COURTMIN", "L_LONGMIN", "Geometry"]]
voies.rename(columns = {'N_SQ_VO':'id2022'}, inplace = True)

#apply preprocessing to voies
voies = preprocess(voies, "L_VOIE", new_colname="street_short")
voies = preprocess(voies, "L_COURTMIN", new_colname="street_abbr")
voies = preprocess(voies, "L_LONGMIN", new_colname="street_long")

In [6]:
#compute prefix dictionary with help of voies data
def get_prefix(row, court, long):
    return row[long].split(row[court])[0]

voies["prefix_court"] = voies.apply(get_prefix, args=("street_short", "street_abbr"), axis=1)
voies["prefix_long"] = voies.apply(get_prefix, args=("street_short", "street_long"), axis=1)
prefix_candidates = dict(zip(voies["prefix_court"], voies["prefix_long"]))
# only get prefixes with . in it
prefix_dict = {key.split(" ")[0]:value.split(" ")[0] for key,value in prefix_candidates.items() if "." in key}

prefix_dict["boul."] = "boulevard"
prefix_dict["boulev."] = "boulevard"
prefix_dict["boulv."] = "boulevard"
prefix_dict["q."] = "quai"
prefix_dict["aven."] = "avenue"
prefix_dict["faub."] = "faubourg"
prefix_dict["fau."] = "faubourg"
prefix_dict["st."] =  "saint"
prefix_dict["impas."] = "impasse"
prefix_dict["l'aub."] = "l'auberge"
prefix_dict["laub"] = "l'auberge"
prefix_dict["st"] = "saint"
prefix_dict["ste"] = "sainte"
prefix_dict["sts"] = "saints"
prefix_dict["nve"] = "neuve"

In [7]:
def get_prefix(street):
    # returns streets which include a "."
    try:
        if "." in street:
            return street
    except:
        print(street)

#### substitute prefixes

In [8]:

      
# get number of abbreviations
print("#abbr. in data before dict substitution:", len(set([get_prefix(street) for street in list(Bottins["rue_processed"])])))
# substitute abbreviations
Bottins["rue_processed"] = substitute_col_by_dict(Bottins["rue_processed"], prefix_dict)
# see how many abbreviations are left
print("#abbr. in data after dict substitution:", len(set([get_prefix(street) for street in list(Bottins["rue_processed"])])))

#abbr. in data before dict substitution: 76549
#abbr. in data after dict substitution: 32913


## manual substitution

#### create manual dictionary

In [40]:
manual_substitution = { 


                "alle. magne": "allemagne",
                "ams. terdam": "amsterdam",
                "av.": "avenue ",
                "av.de": "avenue de ",
                "ay.": "avenue",
                "b. beaumarchais": "boulevard beaumarchais",
                "b. bonne nouv.": "boulevard bonne nouvelle",
                "b. bonne nouvelle": "boulevard bonne nouvelle",
                "b. bonne. nouvelle": "boulevard bonne nouvelle",
                "b. du temple": "boulevard du temple",
                "b. poissonniere": "boulevard poissonniere",
                "boul. ": "boulevard ",
                "boul.": "boulevard ",
                "boul.": "boulevard ",
                "boul.de ": "boulevard de ",
                "bouley. ": "boulevard",
                "bouly.": "boulevard",
                "bourb. villeneuve": "rue bourbon villeneuve",
                "bourbon villen.": "rue bourbon villeneuve",
                "bretonn.": "bretonnerie",
                "carref. ": "carrefour ",
                "ch. d'antin": "rue de la chaussee d'antin",
                "chauss.": "chaussee ",
                "chaussee d'antin": "rue de la chaussee d'antin",
                "chaussee.": "chaussee",
                "che. min": "chemin",
                "cherche midi": "rue du cherche midi",
                "cherche. midi": "rue du cherche midi",
                "dame de": "damede ",
                "dame.": "dame",
                "denazareth": "de nazareth",
                "dutemple": "du temple",
                "echi. quier": "rue de l'echiquier",
                "ecole de med.": "rue de l'ecole de medecine",
                "eglise.": "eglise",
                "f. du temple": "faubourg du temple",
                "f. montmartre": "faubourg montmartre",
                "f. poissoniere": "faubourg poissoniere",
                "f. poissonniere": "faubourg poissonniere",
                "f. saint ": "faubourg saint",
                "faab. " : "faubourg",
                "fanb. ": "faubourg",
                "faub ": "faubourg",
                "faub.. ": "faubourg",
                "faub..": "faubourg ",
                "faub..du": "faubourg du ",
                "faub.": "faubourg ",
                "faub.": "faubourg ",
                "faub.du ": "faubourg du ",
                "faub.montmartre": "faubourg montmartre",
                "faub.poissonniere": "faubourg poissonniere",
                "faub.st.": "faubourg saint",
                "faub.st": "faubourg saint",
                "faub).": "faubourg",
                "faubourg du. temple": "rue du faubourg du temple",
                "faubourg saint an. toine": "rue du faubourg saint antoine",
                "faubourg saint ant.": "rue du faubourg saint antoine",
                "faubourg. ": "faubourg",
                "faubourg..": "faubourg",
                "faubourg.": "faubourg",
                "faubourg.du": "faubourg du",
                "faubourgdu.": "faubourg du ",
                "fauh. ": "faubourg",
                "fauh.. ": "faubourg ",
                "fauh.": "faubourg ",
                "faul.": "faubourg ",
                "faul).": "faubourg",
                "fb. ": "faubourg",
                "fd. poissonniere": "faubourg poissoniere",
                "germain l'aux.": "germain l\'auxerrois",
                "grande.": "grande",
                "haub. ": "faubourg",
                "hauss. mann": "haussmann",
                "houl. ": "boulevard ",
                "impass. ": "impasse",
                "j j. rousseau": "jean jaques rousseau",
                "j. j rousseau": "jean jaques rousseau",
                "j. j.   rousseau": "jean jaques rousseau",
                "j. j.  rousseau": "jean jaques rousseau",
                "j. j. pousseau": "jean jaques rousseau",
                "j. j. r": "jean jaques rousseau", 
                "j. j. rousseau": "jean jaques rousseau",
                "j. j.. rousseau": "jean jaques rousseau",
                "j. j.rousseau": "jean jaques rousseau",
                "j.j. rousseau": "jean jaques rousseau",
                "lafayette": "la fayette",
                "laub. ": "faubourg ",
                "le. compte": "le compte",
                "m. le prince": "rue monsieur le prince",
                "ma. genta": "magenta",
                "mar| tin": "martin",
                "meri. court": "mericourt",
                "mons. le prince": "rue monsieur le prince",
                "mont. martre": "montmartre",
                "montagne sainte gen.": "rue de la montagne sainte genevieve",
                "montm.": "montmartre",
                "montmar. tre": "montmartre",
                "montmart.": "montmartre",
                "n. d ": "notre dame",
                "n. d.":"notre dame",
                "n. da": "notre da",
                "n. de nazareth": "notre dame de nazareth",
                "n.d.": "notre dame",
                "naza. reth": "nazareth",
                "neuve. des petits champs": "rue neuve des petits champs",
                "notre d. de": "notre dame de",
                "notre damede naza. reth": "rue notre dame de nazareth",
                "pass.du": "passage du",
                "pe. tits": "petits",
                "pet. champs": "petits champs",
                "pet. ecuries":"petites ecuries",
                "petites.": "petites ",
                "petitesecuries": "petites ecuries",
                "petits.": "petits",
                "petitschamps": "petits champs",
                "pois. sonniere": "poissonniere",
                "poiss.": "poissonniere",
                "poissonn.": "poissonniere",
                "r.de": "rue de",
                "r.des": "rue des",
                "r.du ": "rue du ",
                "r.st": "rue saint",
                "rambu. teau": "rambouteau",
                "riche. lieu": "richelieu",
                "rue de vaugi. rard": "rue de vaugirard",
                "saint ant.": "saint antoine",
                "saint g. ": "saint germain",
                "saint g.": "saint germain",
                "saint germ.": "saint germain",
                "saint hon.": "saint honore",
                "saintgerm.": "saint germain",
                "saints. peres": "rue des saints peres",
                "se. basaintopol": "sebastopol",
                "sebas. topol": "sebastopol",
                "st.honore": "saint honore",
                "stdenis": "saint denis",
                "stgermain": "saint germain",
                "sthonore": "saint honore",
                "stmartin": "saint martin",
                "stmichel": "saint michel",
                "stras. bourg": "strassbourg",
                "taub. ": "faubourg",
                "tem. ple": "temple",
                "vaugi. rard": "vaugirard",
                "vi. vienne": "vivienne",
                "vil. lette": "villette",
                "vol. taire": "voltaire"

                "n.  d. de": "notre dame de",
                "b. poissonniere": "boulevard poissonniere",
                "ri. voli": "rue de rivoli",
                "croix des petits ch.": "rue croix des petits champs",
                "males. herbes": "malesherbes",
                "poisson. niere": "poissonniere",
                "rous. seau": "rousseau",
                "monsieur le. prince": "monsieur le prince",
                }


#### substitute

In [41]:

# substitute abbreviations
Bottins["rue_processed"] = substitute_col_by_dict(Bottins["rue_processed"], manual_substitution)
print("#abbr. in data after dict substitution:", len(set([get_prefix(street) for street in list(Bottins["rue_processed"])])))

#abbr. in data after dict substitution: 29242


#### show streets with remaining prefixes

In [42]:
Counter([x for x in Bottins["rue_processed"] if "." in x]).most_common()

[('gren. saint germain', 128),
 ('boulevard de saintras. bourg', 80),
 ('grenelle saint h.', 64),
 ('paradis. poissonniere', 58),
 ('grenelle. saint germain', 48),
 ('ferme des math.', 46),
 ('angouleme du t.', 43),
 ('michel le. comte', 43),
 ('saint domin. saint germain', 40),
 ('gren. saint honore', 39),
 ('neuve. saint augusaintin', 39),
 ('neuve. saint merri', 38),
 ('bonl. du temple', 37),
 ('la. fayette', 35),
 ('simon le. franc', 34),
 ('gra. villiers', 33),
 ('saint hyacinthe saint m.', 33),
 ('monsieur le. prince', 33),
 ('neuve. popincourt', 33),
 ('boulevard ri. chard lenoir', 33),
 ('bonl. poissonniere', 32),
 ('saint de. nis', 32),
 ('marais saint m.', 32),
 ('ro. quette', 32),
 ('notre damede . nazareth', 32),
 ('faubourg saint de. nis', 31),
 ('boulevard bonne. nouvelle', 31),
 ('n d. de nazareth', 31),
 ('vieille du. temple', 31),
 ('s. denis', 30),
 ('boulevard des batignol. les', 30),
 ('boal. beaumarchais', 29),
 ('bonl. saint martin', 29),
 ('vieille. du temple', 2

#TODO hier noch letztes Preprocessing, wo mehrere Leerzeichen zu einem gemacht werden

## data where only 

In [11]:
streets = pd.read_pickle("data/FinalUnique.pkl")

In [12]:
# group streets based on their short name
grouped_streets = streets.groupby("name_prep", as_index=False).agg({"streetname_prep": list})
# split streets in those that are unique and those that aren't
unique_short_streets = grouped_streets[grouped_streets['streetname_prep'].str.len() == 1]
unique_short_streets[["streetname_prep"]] = unique_short_streets[["streetname_prep"]].apply(lambda x: x[0])
multiple_short_streets = grouped_streets[grouped_streets['streetname_prep'].str.len() > 1]

print(f"#streets with unique short streetname: {len(unique_short_streets)}, not unique: {len(multiple_short_streets)}")


#streets with unique short streetname: 5374, not unique: 780


C:\Users\Ben\AppData\Local\Temp\ipykernel_2288\1396352598.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_short_streets[["streetname_prep"]] = unique_short_streets[["streetname_prep"]].apply(lambda x: x[0])


In [13]:
multiple_short_streets.head(5)

,name_prep,streetname_prep
18,abbaye,"[chemin de l'abbaye, rue de l'abbaye]"
35,abbesses,"[passage des abbesses, place des abbesses, rue..."
50,acacias,"[passage des acacias, petite rue des acacias]"
73,adrienne,"[cite adrienne, villa adrienne]"
110,albert bartholome,"[avenue albert bartholome, square albert barth..."


# TODO check overlapping geodata streets (above)
multiple_short_streets: same short streetname, check for overlaping geodata in streetname

In [1]:
unique_short_streets.head(5)

NameError: name 'unique_short_streets' is not defined

## Save preprocessed datasets

In [15]:
Bottins.to_pickle("data/bottins_prep.pkl")
unique_short_streets.to_pickle("data/unique_short_streets.pkl")
multiple_short_streets.to_pickle("data/not_unique_short_streets.pkl")